In [1]:
import pandas as pd
import json
import clips

## Load & Pre-Process Dataset

In [2]:
data_df = pd.read_csv('../data/raw/amazon_popular_books_dataset.csv', usecols=['title', 'categories', 'rating', 'reviews_count'])

# Pre-process Dataset
data_df['rating'] = pd.to_numeric(data_df['rating'].str.slice(0, 3))

# Add weighted rating
m = 100  # set the minimum number of reviews to 100
C = data_df['rating'].mean()  # calculate the mean rating across all books

# define a function to calculate the weighted rating
def weighted_rating(row):
    v = row['reviews_count']
    R = row['rating']
    return (v / (v+m)) * R + (m / (v+m)) * C

# apply the function to each row of the dataframe to calculate the weighted rating
data_df['weighted_rating'] = data_df.apply(weighted_rating, axis=1)

# Remove "Books" entry from categories
data_df['categories'] = data_df['categories'].apply(lambda categories: json.loads(categories)[1:])

data_df

,rating,reviews_count,title,categories,weighted_rating
0,4.6,13451,Wuthering Heights (Collins Classics),"[Literature & Fiction, Genre Fiction]",4.600165
1,4.8,16628,THE DAYS THE CRAYONS QUIT,"[Children's Books, Literature & Fiction]",4.798938
2,4.8,11275,War Lord: Book 13 (The Last Kingdom Series),"[Literature & Fiction, Genre Fiction]",4.798438
3,4.8,15520,Code Name Bananas: The hilarious and epic new ...,"[Children's Books, Literature & Fiction]",4.798863
4,4.8,10884,Skincare: The award-winning ultimate no-nonsen...,"[Crafts, Hobbies & Home, Home Improvement & De...",4.798383
...,...,...,...,...,...
2264,4.2,17923,Unspeakable Things,"[Mystery, Thriller & Suspense, Thrillers & Sus...",4.202343
2265,4.8,25304,What to Expect When You’re Expecting,"[Health, Fitness & Dieting, Women's Health, Pr...",4.799301
2266,4.7,11040,The Home Edit: A Guide to Organizing and Reali...,"[Crafts, Hobbies & Home, Home Improvement & De...",4.699303
2267,4.4,28030,The Family Upstairs: A Novel,"[Mystery, Thriller & Suspense, Thrillers & Sus...",4.400790


## Choose 10 Random Book Titles

In [54]:
random_10_rows = data_df.sample(n=10)
random_10_titles = random_10_rows['title'].tolist()
random_10_titles

['The Song of Achilles',
 'The Obesity Code: Unlocking the Secrets of Weight Loss',
 'The Book of Longings: A Novel',
 'Die Trying: Jack Reacher, Book 2',
 'Green Eggs and Ham',
 'The Day the Crayons Quit',
 'The Haunting of Brynn Wilder: A Novel',
 'Untamed',
 'Thinking, Fast and Slow',
 'The One and Only Ivan']

## Get Unique Categories

In [55]:
categories = data_df.explode('categories')['categories'].unique()

## Create CLIPS environment

In [57]:
env = clips.Environment()

# Book Template
BOOK_TEMPLATE_STRING = """
(deftemplate book
    (slot title (type STRING))
    (slot category (type STRING))
    (slot w_rating (type FLOAT)))
"""
env.build(BOOK_TEMPLATE_STRING)

# Categorized Book Template
CATEGORIZED_BOOK_TEMPLATE_STRING = """
(deftemplate categorized-book
    (slot title (type STRING))
    (slot category (type STRING)))
"""
env.build(CATEGORIZED_BOOK_TEMPLATE_STRING)

# Uncategorized Book Template
UNCATEGORIZED_BOOK_TEMPLATE_STRING = """
(deftemplate uncategorized-book
    (slot title (type STRING)))
"""
env.build(UNCATEGORIZED_BOOK_TEMPLATE_STRING)

# Recommended Book Template
RECOMMENDED_BOOK_TEMPLATE_STRING = """
(deftemplate recommended-book
    (slot title (type STRING))
    (slot w_rating (type FLOAT)))
"""
env.build(RECOMMENDED_BOOK_TEMPLATE_STRING)

# Category Counter Template
CATEGORY_COUNTER_TEMPLATE_STRING = """
(deftemplate category-counter
    (slot category (type STRING))
    (slot count (type INTEGER)))
"""
env.build(CATEGORY_COUNTER_TEMPLATE_STRING)

# Categorized Books Rule
CATEGORIZED_BOOKS_RULE_STRING = """
(defrule categorize-books
    "Categorize books that have a category."
    (declare (salience 50))
    (book (title ?title) (category ?category))
    =>
    (assert (categorized-book (title ?title) (category ?category))))
"""
env.build(CATEGORIZED_BOOKS_RULE_STRING)

# Count Category Rule
COUNT_CATEGORY_RULE_STRING = """
(defrule count-category
    "Counter for each category."
    (declare (salience 40))
    ?categorized-book-adr <- (categorized-book (title ?title) (category ?category))
    ?category-counter-adr <- (category-counter (count ?count) (category ?category-title&:(eq ?category-title ?category)))
    ?uncategorized-book-adr <- (uncategorized-book (title ?uncategorized-title&:(eq ?uncategorized-title ?title)))
    =>
    (retract ?categorized-book-adr ?category-counter-adr)
    (assert (category-counter (category ?category) (count (+ ?count 1)))))
"""
env.build(COUNT_CATEGORY_RULE_STRING)

# Highest Category Count Rule
HIGHEST_CATEGORY_COUNT_RULE_STRING = """
(defrule highest-category-count
    "Find the category with the highest count."
    (declare (salience 30))
    (category-counter (count ?n1) (category ?category))
    (not (category-counter (count ?n2&:(> ?n2 ?n1))))
    =>
    (assert (highest-category ?category)))
"""
env.build(HIGHEST_CATEGORY_COUNT_RULE_STRING)

# Rule that picks only one "highest-category"
PICK_ONE_HIGHEST_CATEGORY_RULE_STRING = """
(defrule pick-one-highest-category
    "Picks only one 'highest category', just in case there are two categories with the same count."
    (declare (salience 20))
    (highest-category ?category)
    (not (highest-category-picked))
    =>
    (assert (most-popular-category ?category))
    (assert (highest-category-picked)))
"""
env.build(PICK_ONE_HIGHEST_CATEGORY_RULE_STRING)

# Return Recommended Books Rule
RETURN_RECOMMENDED_BOOKS_RULE_STRING = """
(defrule return-recommended-books
    "Return recommended books."
    (declare (salience 10))
    (book (title ?title) (category ?category) (w_rating ?w_rating))
    (most-popular-category ?most-popular-category&:(eq ?most-popular-category ?category))
    =>
    (assert (recommended-book (title ?title) (w_rating ?w_rating))))
"""
env.build(RETURN_RECOMMENDED_BOOKS_RULE_STRING)

book_template = env.find_template('book')
uncategorized_book_template = env.find_template('uncategorized-book')
category_counter_template = env.find_template('category-counter')

# Assert each category and set its count to 0
for category in categories:
    category_counter_template.assert_fact(category=category, count=0)

# Assert each book and its respective category
for _, row in data_df.iterrows():
    for category in row['categories']:
        book_template.assert_fact(
            title=row['title'],
            category=category,
            w_rating=row['weighted_rating']
        )

for title in random_10_titles:
    uncategorized_book_template.assert_fact(title=title)

env.run()

LookupError: DefinedFacts 'most-popular-category' not found